In [5]:
# imports and main data files

"""
Initialization
"""
# imports
import os
import pandas as pd
from collections import Counter
import time
from math import log10
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from collections import defaultdict
from scipy import stats
import matplotlib.ticker as mticker
import numpy as np
import seaborn as sns
import sys

plt.rcParams.update({'font.size': 15})

# constant, data
os.chdir("/Users/antata/Library/CloudStorage/OneDrive-BaylorCollegeofMedicine/text-mining/categories/")
category_names = ["obesity", "cancer", "cardiovascular", "digestive", "endocrine", "hematological", "immune", "metabolic", "neurological", "urogenital", "respiratory"]#, "T2D"]
color_template = ['#800000','#e6194B','#f58231','#f3c300','#469990','#808000','#2f8e3b','#0db7dd','#4363d8','#911eb4','#f032e6']#, "#000075"]#'#8298e5'
controls = []
corsiv_probe_df = pd.read_csv("../control/corsiv_all_probes_id.txt", sep="\t", names=["chr", "start", "end", "probeId", "corsiv_start", "corsiv_end", "corsiv_id"])
corsiv_probe_list = set(corsiv_probe_df.iloc[:,3])
for i in range(1, 11):
    control_probe_df = pd.read_csv(f"../control/control_probes_{i}.txt", sep="\t", names=["chr", "start", "end", "probeId", "_", "corsiv_start", "corsiv_end", "corsiv_id"])
    control_probe_list = set(control_probe_df.iloc[:,3])
    controls.append(control_probe_list)
control_probe_df = pd.read_csv("../control/control_all_probes_id.txt", sep="\t", names=["chr", "start", "end", "probeId", "id"])
control_probe_list = set(control_probe_df.iloc[:,3])

epic = pd.read_csv("../humanData/database/EPIC.hg38.txt", sep="\t", header=None)
epic_probe_list = set(epic.iloc[:,3])
hm450 = pd.read_csv("../humanData/database/HM450.hg38.txt", sep="\t", header=None)
hm450_probe_list = set(hm450.iloc[:,3])
illumina = epic_probe_list.union(hm450_probe_list)
enrichment_cutoff = (len(corsiv_probe_list) / len(illumina)*1.5)
enrichment_cutoff
epic_hm450_probe = pd.read_csv("../control/humanData/EPIC_HM450.clean.bed", sep="\t", names=["chr", "start", "end", "probeId"])
corsiv_genes = pd.read_csv("../control/corsiv_all_genes.csv")
corsiv_genes = corsiv_genes[corsiv_genes["id"].isin(corsiv_probe_df["corsiv_id"])]
corsiv_genes.fillna("", inplace=True)
control_genes = pd.read_csv("../control/control_all_genes.csv")
control_genes = control_genes[control_genes["id"].isin(control_probe_df["id"])]
control_genes.fillna("", inplace=True)

In [14]:
# read in probes

"""
read all probes if combined file alrdy exists
"""
cat_probes_dict = []
# valid_studies = pd.read_csv("pubmed_search/valid_studies.csv")
def read_in_probes(input_cat):
    if input_cat == "metabolic":
        df = pd.read_csv(f"probe/metabolic_diseases_all_probes.csv")
    else:
        df = pd.read_csv(f"probe/{input_cat}_all_probes.csv")
    df.drop_duplicates(subset=["pmcid", "probeId"], inplace=True)
    probe_list = df["probeId"].to_list()
    c = dict(Counter(probe_list))
    return c


for cat in category_names:
    start = time.time()
    print(cat)
    # if cat != "anthropometric":
    #     continue
    c = read_in_probes(cat)
    cat_probes_dict.append(c)
    end = time.time()
    print(f'Time for {cat} code to run: ', end - start)


obesity
Time for obesity code to run:  0.02238917350769043
cancer
Time for cancer code to run:  0.17228388786315918
cardiovascular
Time for cardiovascular code to run:  0.023622989654541016
digestive
Time for digestive code to run:  0.0371401309967041
endocrine
Time for endocrine code to run:  0.03746509552001953
hematological
Time for hematological code to run:  0.02130579948425293
immune
Time for immune code to run:  0.04654693603515625
metabolic
Time for metabolic code to run:  0.050930023193359375
neurological
Time for neurological code to run:  0.11426830291748047
urogenital
Time for urogenital code to run:  0.1017148494720459
respiratory
Time for respiratory code to run:  0.03491997718811035


In [7]:
for c in cat_corsiv_dict:
    print(max(c.values()))

5
15
4
6
7
3
7
7
13
8
4


In [ ]:
import math
m = 0
for c in cat_probes_dict:
    m = max(m, max(c.values()))

cat_col = []
paper_threshold_col = []
probes_num_col = []
unique_probes_num_col = []
probes_num_col2 = []
ratio_col = []
for i in range(1, m+1):
    curr_probes_dict = []
    for j, c in enumerate(cat_probes_dict):
        c = {k:v for k, v in c.items() if v >= i}
        cat_col.append(category_names[j])
        probes_num_col.append(len(c))
        curr_probes_dict.append(c)
    df = pd.DataFrame(curr_probes_dict)
    df = df.fillna(0).transpose()
    df.columns = category_names
    for j, c in enumerate(cat_probes_dict):
        unique_probes = set(df[(df[category_names[j]] != 0) & (df.drop(columns=[category_names[j]]) == 0).all(axis=1)].index)
        total_probes = set(df[(df[category_names[j]] != 0)].index)
        probes_num_col2.append(len(total_probes))
        unique_probes_num_col.append(len(unique_probes))
        r = round(len(unique_probes) / len(total_probes), 3) if len(total_probes) > 0 else math.nan
        ratio_col.append(r*100)
    paper_threshold_col += [i] * len(category_names)
# probes_num_col == probes_num_col2
# print(len(cat_col), len(paper_threshold_col), len(probes_num_col), len(unique_probes_num_col), len(probes_num_col2), len(ratio_col))
res = pd.DataFrame({"Categories":cat_col, "paper num":paper_threshold_col, "Total probes": probes_num_col, "Unique probes": unique_probes_num_col, "ratio%": ratio_col})
res  

In [ ]:

# Assuming res is already defined with the relevant columns.
# Create the figure and axis objects
# fig, ax = plt.subplots(figsize=(10, 6))

# Loop over each category
for i, cat in enumerate(category_names):
    group = res[(res["Categories"]==cat) & (~res["ratio%"].isna())]
    fig, ax = plt.subplots(figsize=(10, 6))
    # Plot the line for each category, NaNs will break the line automatically
    ax.plot(group["paper num"], group["ratio%"], label=category_names[i], marker="o", color=color_template[i])

    # Set y-axis range from 0 to 100% and bin at 20%
    ax.set_ylim(0, 100)
    ax.set_yticks(range(0, 101, 20))  # y-ticks at every 20%
    ax.set_xticks(range(0, int(max(group["paper num"]))+1, 5))

    # Label the axes
    ax.set_xlabel("Number of Papers Reporting Probes")
    ax.set_ylabel("Unique to All Probes Ratio (%)")

    # Add title
    ax.set_title(category_names[i])

    # Add legend
    # ax.legend(title="Categories")

    # Show the plot
    plt.savefig(f"/Users/antata/Desktop/text-mining-figures/final/stats/{category_names[i]}_ratio.jpeg")
    plt.close()


In [20]:
import seaborn as sns
import sys
# sys.setrecursionlimit(100000)
# go heatmap for all probes by category
paper_threshold = 2
heatmap_df = []

for c in cat_probes_dict:
    c = {k:v for k, v in c.items() if v >= paper_threshold}
    heatmap_df.append(c)

heatmap_df = pd.DataFrame(heatmap_df)
heatmap_df = heatmap_df.fillna(0).transpose()
heatmap_df.columns = category_names
# gr = sns.light_palette("red", as_cmap=True)
# ax = sns.clustermap(heatmap_df, cmap=gr, cbar_kws=dict(ticks=[0, 10, 20, 30, 40, 50]), yticklabels=False)
# plt.setp(ax.ax_heatmap.get_xticklabels(), rotation=45)# ax.cax.set_visible(False)
# ax.figure.axes[-1].set_ylabel('Number of Papers')
heatmap_df.to_csv(f"heatmap/probe_based_heatmap_{paper_threshold}papers_0107.csv")

In [21]:
heatmap_df

,obesity,cancer,cardiovascular,digestive,endocrine,hematological,immune,metabolic,neurological,urogenital,respiratory
cg12484113,3.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
cg08548559,4.0,1.0,3.0,1.0,2.0,0.0,1.0,2.0,2.0,0.0,2.0
cg26663590,7.0,2.0,2.0,3.0,3.0,1.0,1.0,4.0,2.0,0.0,2.0
cg07136133,5.0,0.0,1.0,0.0,3.0,0.0,0.0,3.0,0.0,2.0,0.0
cg18120259,4.0,0.0,4.0,2.0,2.0,0.0,1.0,5.0,2.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
cg06247406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
cg24753760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
cg02065151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
cg10411339,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# column stats
def generate_stats(df, cat, verbal=True):
    probe_nums = []
    max_probes = []
    for i in range(len(cat)):
        curr_probes = df[df[cat[i]]>0]
        if curr_probes.empty:
            if verbal:
                print(f"{cat[i]}: No probes showing up in ≥ {paper_threshold} papers")
            else:
                probe_nums.append(0)
                max_probes.append(0)
            continue
        probe_num = curr_probes.shape[0]
        max_probe = int(curr_probes.iloc[:, i].max())
        if verbal:
            print(f"{cat[i]}: {probe_num} probes in ≥ {paper_threshold} papers, max number of papers: {max_probe} papers")
        else:
            probe_nums.append(probe_num)
            max_probes.append(max_probe)
    if not verbal:           
        stat_df = pd.DataFrame({"Categories": cat, "Probe Nums": probe_nums, "Max paper": max_probes})
        return stat_df
    return 0
generate_stats(heatmap_df, category_names, False)

In [ ]:
# neurological categories

# Initialize an empty dictionary to store the results
mesh_map = defaultdict(set) #term:code
mesh_tree = {} #big category:set of subcategories
file_path = '../humanData/database/mtrees2024.txt'

# Read the lines from the file
with open(file_path, 'r') as file:
    for line in file:
        # Split each line into A and B
        parts = line.strip().split(';')
        if len(parts) == 2:
            term, code = parts
            mesh_map[term].add(code)
        else:
            print(parts)
            
def starts_with_any(given_string, string_list):
    for prefix in string_list:
        if given_string.startswith(prefix):
            return True
    return False

def filter_mesh_list(input):
    return any(input in sublist for sublist in mesh_tree.values())

keywords = ["Mental Disorders", "Nervous System Diseases"]

for kw in keywords:
    mesh_tree[kw] = set([k for k, v in mesh_map.items() for c in v if starts_with_any(c, mesh_map[kw])])


In [ ]:
neurological_df = pd.read_csv("probe/neurological_all_probes.csv")
neurological_df["Filtered Mesh Term"] = neurological_df["Filtered Mesh Term"].apply(lambda x: [term.strip() for term in x.split("|")])

In [ ]:

plt.rcParams.update({'font.size': 10})

temp = neurological_df
temp1 = temp.drop_duplicates(subset="pmcid")
mesh_count_by_study = defaultdict(int)
mesh_terms = list(temp1["Filtered Mesh Term"])
pmcids = list(temp1["pmcid"])
term_pmcid_map = defaultdict(set)
for i in range(len(mesh_terms)):
    m = mesh_terms[i]
    # terms = [term.strip() for term in m.split("|")]
    for t in m:
        mesh_count_by_study[t] += 1
        term_pmcid_map[t].add(pmcids[i])
mesh_count_by_study = {key:count for key, count in mesh_count_by_study.items() if filter_mesh_list(key) and count >= 14}
print(len(mesh_count_by_study))
mesh_count_by_study = dict(sorted(mesh_count_by_study.items(), reverse=True))
categories = list(mesh_count_by_study.keys())
counts = list(mesh_count_by_study.values())
# sig_categories = ["Schizophrenia Spectrum and Other Psychotic Disorders", "Schizophrenia", "Neurologic Manifestations", "Neurobehavioral Manifestations","Nervous System Diseases","Mental Disorders","Intellectual Disability"]
# Plotting the horizontal histogram
plt.figure(figsize=(8,8))
bars = plt.barh(categories, counts, color=color_template[8])
for bar, value in zip(bars, counts):
    plt.text(bar.get_width()+15, bar.get_y() + bar.get_height()/2, value, ha='center', va='center')
plt.ylim(-0.5, len(categories))
plt.xlabel('Number of Papers')
plt.ylabel('MeSH Categories')
plt.title('Distribution of Neurological Papers by MeSH Categories, papers ≥ 14')
plt.tight_layout()
# plt.savefig("figures/cancer_paper_by_mesh_>=10.jpeg", format="jpeg", bbox_inches="tight")
plt.show()

# mesh_count_by_study = plot_categories("neurological")

In [ ]:
paper_threshold = 3
def get_neurological_probe(disease):
    df_categorized = temp[temp["pmcid"].isin(set().union(*[term_pmcid_map[d] for d in disease]))]
    probe_list = df_categorized["probeId"].to_list()
    probe_list = [s for s in probe_list if (str(s).startswith("cg") or str(s).startswith("ch."))]
    c = Counter(probe_list)
    c = {key:count for key, count in c.items() if (key in epic_probe_list) or (key in hm450_probe_list)}
    c = {key:count for key, count in c.items() if count >= paper_threshold}
    return c

neuro_heatmap = []
neuro_categories = list(mesh_count_by_study.keys())
for term in neuro_categories:
    probes = get_neurological_probe([term])
    neuro_heatmap.append(probes)

neuro_heatmap = pd.DataFrame(neuro_heatmap)
neuro_heatmap = neuro_heatmap.fillna(0).transpose()
neuro_heatmap.columns = neuro_categories
neuro_heatmap
# gr = sns.light_palette("red", as_cmap=True)
# ax = sns.clustermap(heatmap_df, cmap=gr, cbar_kws=dict(ticks=[0, 10, 20, 30, 40, 50]), yticklabels=False)
# plt.setp(ax.ax_heatmap.get_xticklabels(), rotation=45)# ax.cax.set_visible(False)
# ax.figure.axes[-1].set_ylabel('Number of Papers')
neuro_heatmap.to_csv(f"heatmap/neuro_probe_based_heatmap_{paper_threshold}papers.csv")

In [ ]:
df = generate_stats(neuro_heatmap, neuro_categories, False)
df.sort_values(by="Probe Nums", ignore_index=True)

In [ ]:
# column stats
df = generate_stats(neuro_heatmap, neuro_categories, False)
target = df[(df["Probe Nums"]>=10) & (df["Probe Nums"]<= 300)]["Categories"] # & (df["Probe Nums"]<= 300)
target_df = neuro_heatmap[target]
df_no_zero_rows = target_df[(target_df != 0).any(axis=1)]
df_no_zero_rows.to_csv("heatmap/neuro_probe_based_heatmap_10-300probes.csv")

In [ ]:
# get all genes related to corsivs that were covered on illumina
corsiv_illumina_overlap = set(corsiv_probe_df[corsiv_probe_df["probeId"].isin(hm450_probe_list) | corsiv_probe_df["probeId"].isin(epic_probe_list)]["corsiv_id"])
gene_names = corsiv_genes[corsiv_genes["id"].isin(corsiv_illumina_overlap)]["gene_name"]
res = []
for names in gene_names:
    names = names.split(";")
    res += names
corsiv_illumina_genes = set(res)
corsiv_illumina_genes.discard('')
# target_libraries = ["Jensen_DISEASES", "DisGeNET", "GO_Biological_Process_2023", "Human_Phenotype_Ontology"]
# run_enrichr(corsiv_illumina_genes, 1, target_libraries, f"go_probe/go_results/go_corsiv_illumina_overlap.csv")

# # get all genes related to control that were covered on illumina
control_illumina_overlap = set(control_probe_df[control_probe_df["probeId"].isin(hm450_probe_list) | control_probe_df["probeId"].isin(epic_probe_list)]["id"])
gene_names = control_genes[control_genes["id"].isin(control_illumina_overlap)]["gene_name"]
res = []
for names in gene_names:
    names = names.split(";")
    res += names
control_illumina_genes = set(res)
control_illumina_genes.discard('')
# target_libraries = ["Jensen_DISEASES", "DisGeNET", "GO_Biological_Process_2023", "Human_Phenotype_Ontology"]
# run_enrichr(control_illumina_genes.union(corsiv_illumina_genes), 1, target_libraries, f"go_probe/go_results/go_corsiv_control_illumina_overlap.csv")
background_genes = control_illumina_genes.union(corsiv_illumina_genes)


In [ ]:
len(corsiv_illumina_genes), len(control_illumina_genes), len(background_genes)

In [ ]:
e = pd.read_csv("../humanData/EPIC_manifest_cleaned.csv")
hm = pd.read_csv("../humanData/HM450_manifest_cleaned.csv")
ref = pd.concat([e, hm])
ref["Unique Gene"] = ref["UCSC_RefGene_Name"].apply(lambda x: ";".join(set(x.split(";"))) if isinstance(x, str) else "")
ref.drop_duplicates(inplace=True)
ref = ref[["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID", "Unique Gene"]]

def group_probes_GREAT(cat):
    df = pd.read_csv(f"go_probe/{cat}_3papers_unique_all.txt", sep="\t", names=["chr", "start", "end", "probe_id"])
    df = ref[ref["Probe_ID"].isin(df["probe_id"])]
    df = df.sort_values(by=['Probe_Start', 'Probe_End'])

    chr_order = {f'chr{i}': i for i in range(1, 23)}
    df = df.sort_values(
        by=['Probe_Chr', 'Probe_Start'],
        key=lambda x: x.map(chr_order) if x.name == 'Probe_Chr' else x
    )

    # Assign a unique index to consecutive rows sharing the same Unique Gene
    df['gene_group'] = ((df['Unique Gene'] != df['Unique Gene'].shift()) | (df['Probe_Chr'] != df['Probe_Chr'].shift())).cumsum()
    grouped = df.groupby("gene_group")
    # # Aggregate the grouped data
    df_grouped = grouped.agg({
        'Probe_Chr': 'first',
        'Probe_Start': 'min',
        'Probe_End': 'max',
        'Probe_ID': lambda x: ';'.join(x),
        'Unique Gene': 'first',
    })
    df_grouped = df_grouped.reset_index(drop=True)
    df_grouped = df_grouped.sort_values(
        by=['Probe_Chr', 'Probe_Start'],
        key=lambda x: x.map(chr_order) if x.name == 'Probe_Chr' else x
    )
    df_grouped[["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID"]].to_csv(f"go_probe/GREAT_{cat}.txt", index=0, sep="\t", header=0)

In [ ]:
for cat in category_names:
    group_probes_GREAT(cat) 

In [22]:
import requests, json, math

e = pd.read_csv("../humanData/EPIC_manifest_cleaned.csv")
hm = pd.read_csv("../humanData/HM450_manifest_cleaned.csv")
ref = pd.concat([e, hm])
ref.drop_duplicates(inplace=True)


target_libraries = ["Jensen_DISEASES", "DisGeNET", "GO_Biological_Process_2023",  "GWAS_Catalog_2023", "Human_Gene_Atlas", "Human_Phenotype_Ontology"] #"GO_Cellular_Component_2023", "GO_Molecular_Function_2023",

def get_gene_set(target_id, gene_num=None):
    # if not gene_num:
        # corsiv_overlap_probes = set(count_dictionary.keys()).intersection(corsiv_probe_list)
        # corsivs = set(corsiv_probe_df[corsiv_probe_df["probeId"].isin(corsiv_overlap_probes)]["corsiv_id"])
    gene_names = corsiv_genes[corsiv_genes["id"].isin(target_id)]["gene_name"]
    res = []
    for names in gene_names:
        names = names.split(";")
        res += names
    res = set(res)
    res.discard('')
    return res, len(target_id)

def get_gene_illumina(target_id):
    tmp = ref[ref["Probe_ID"].isin(target_id)]
    res = []
    for names in tmp["UCSC_RefGene_Name"]:
        if isinstance(names, float) and math.isnan(names):
            continue
        names = names.split(";")
        res += names
    res = set(res)
    res.discard('')
    return res, len(target_id)

def run_enrichr(input_genes, pval=0.05, input_libraries=target_libraries, output_fp=None):
    if len(input_genes) == 0:
        return
    # input genes
    ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/addList'
    genes_str = '\n'.join(input_genes)
    description = 'Gene list for obesity'
    payload = {
        'list': (None, genes_str),
        'description': (None, description)
    }
    response = requests.post(ENRICHR_URL, files=payload)
    if not response.ok:
        raise Exception('Error analyzing gene list')
    data = json.loads(response.text)
    
    # get results for each library
    ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/enrich'
    query_string = '?userListId=%s&backgroundType=%s'
    user_list_id = data["userListId"]
    res = []
    for lib in input_libraries:
        gene_set_library = lib
        response = requests.get(
            ENRICHR_URL + query_string % (user_list_id, gene_set_library)
        )
        if not response.ok:
            raise Exception('Error fetching enrichment results')
        data = json.loads(response.text)[lib]
        data = [r for r in data if r[6]<pval]
        temp = pd.DataFrame(data, columns=["Index", "Name", "P-value", "Odds Ratio", "Combined score", "Genes", "Adjusted p-value", "unknown1", "unknown2"])
        temp = temp[["Name", "P-value", "Adjusted p-value", "Odds Ratio", "Combined score", "Genes"]]
        temp["Adjusted p-value"] = temp["Adjusted p-value"].apply(lambda x: round_to_significant_figures(x, 3))
        temp["Odds Ratio"] = temp["Odds Ratio"].apply(lambda x: round_to_significant_figures(x, 3))
        temp["Combined score"] = temp["Combined score"].apply(lambda x: round_to_significant_figures(x, 3))
        temp["P-value"] = temp["P-value"].apply(lambda x: round_to_significant_figures(x, 3))
        temp["Genes"] = temp["Genes"].apply(lambda x: "; ".join(x))
        temp["Database"] = lib
        temp.sort_values(by="Adjusted p-value", ascending=True, inplace=True)
        res.append(temp)
    result_df = pd.concat(res, ignore_index=True)

    if output_fp:
        result_df.to_csv(output_fp, index=0)
    return result_df

# import numpy as np
def round_to_significant_figures(x, sig):
    if x == 0:
        return 0
    else:
        return round(x, sig-int(np.floor(np.log10(abs(x))))-1)
    
def run_enrichr_with_background(input_genes, background_genes, pval, input_libraries, output_fp=None):
    base_url = "https://maayanlab.cloud/speedrichr"
    description = ''
    # input test gene list
    res = requests.post(
        base_url+'/api/addList',
        files=dict(
        list=(None, '\n'.join(input_genes)),
        description=(None, description),
        )
    )
    if res.ok:
        userlist_response = res.json()
    else:
        raise Exception("Error adding gene list.")
        
    # input background gene list
    res = requests.post(
        base_url+'/api/addbackground',
        data=dict(background='\n'.join(background_genes)),
    )

    if res.ok:
        background_response = res.json()
    else:
        raise Exception("Error adding background list.")
    result_df = []
    # get enrichment results
    for lib in input_libraries:
        res = requests.post(
                base_url+'/api/backgroundenrich',
                data=dict(
                userListId=userlist_response["userListId"],
                backgroundid=background_response["backgroundid"],
                backgroundType=lib,
                )
            )
        if not res.ok:
            raise Exception("Error analyzing data.")
        # else:
            # print(f"{lib} done.")
        data = json.loads(res.text)[lib]
        data = [r for r in data if r[6]<pval]
        temp = pd.DataFrame(data, columns=["Index", "Name", "P-value", "Odds Ratio", "Combined score", "Genes", "Adjusted p-value", "unknown1", "unknown2"])
        temp = temp[["Name","P-value", "Odds Ratio", "Combined score", "Genes", "Adjusted p-value", "Genes"]]
        temp["Adjusted p-value"] = temp["Adjusted p-value"].apply(lambda x: round_to_significant_figures(x, 3))
        temp["Genes"] = temp["Genes"].apply(lambda x: "; ".join(x))
        temp["Database"] = lib
        result_df.append(temp)
    result_df = pd.concat(result_df, ignore_index=True)
    if output_fp:
        if not result_df.empty:
            result_df.to_csv(output_fp, index=0)
        else:
            print(f"No significant result for {output_fp}")
    return result_df



In [ ]:
# go on neuro probes
# paper_threshold = 3
cats =[]
thresh = []
gnum = []
pnum = []
for cat in neuro_categories:
    target_category = cat
    df = pd.read_csv(f"heatmap/neuro_probe_based_heatmap_10-300probes.csv", index_col=0)
    if target_category not in df.columns:
        continue
    unique_probes = set(df[(df[target_category] != 0) & (df.drop(columns=[target_category]) == 0).all(axis=1)].index)

    g, l = get_gene_illumina(unique_probes)
    print(f"{cat} ≥ {paper_threshold} papers: {len(g)} genes associated with {l} probes")
    cats.append(cat)
    thresh.append(paper_threshold)
    gnum.append(len(g))
    pnum.append(l)
    # run_enrichr(g, output_fp=f"go_probe/go_results/{target_category}_all_{paper_threshold}papers_unique.csv")
output_df = pd.DataFrame({"Categories": cats, "Paper cutoff": thresh, "Number of Genes": gnum, "Number of Unqiue Probes": pnum})

In [ ]:
len(corsiv_probe_list), len(control_probe_list)

In [ ]:
df = pd.read_csv("heatmap/probe_based_heatmap_3papers.csv", index_col=0)
target_category="obesity"
df[(df[target_category] != 0) & (df.drop(columns=[target_category]) == 0).all(axis=1)]


In [ ]:
unique_probes = set(df[(df[target_category] != 0) & (df.drop(columns=[target_category]) == 0).all(axis=1)].index)
ddf = epic_hm450_probe[epic_hm450_probe['probeId'].isin(unique_probes)]
ddf[ddf.duplicated(subset="probeId", keep=False)]

In [ ]:
paper_threshold = 3
df = pd.read_csv(f"heatmap/probe_based_heatmap_{paper_threshold}papers.csv", index_col=0)
# for cat in category_names:
#     target_category = cat
#     unique_probes = set(df[(df[target_category] != 0) & (df.drop(columns=[target_category]) == 0).all(axis=1)].index)
#     l1 = len(unique_probes)
#     l2 = len(unique_probes.intersection(corsiv_probe_list))
#     print(cat, l1, l2, l2/l1)
print(df[df.index.isin(corsiv_probe_list)].shape[0], df.shape[0])
print(df[df.index.isin(corsiv_probe_list)].shape[0] / df.shape[0])

In [ ]:
paper_threshold = 3
df = pd.read_csv(f"heatmap/probe_based_heatmap_{paper_threshold}papers.csv", index_col=0)
for cat in category_names:
    target_category = cat
    # all probes unique to each category
    unique_probes = set(df[(df[target_category] != 0) & (df.drop(columns=[target_category]) == 0).all(axis=1)].index)
    probe_output_df = epic_hm450_probe[epic_hm450_probe['probeId'].isin(unique_probes)]
    unique_probes
    print(len(unique_probes), probe_output_df.shape[0])
    probe_output_df.to_csv(f"go_probe/{target_category}_{paper_threshold}papers_unique_all.txt", sep="\t", header=0, index=0)

In [27]:
# go on probes
paper_threshold = 1
cats =[]
thresh = []
gnum = []
pnum = []
cnum = []
df = pd.read_csv(f"heatmap/probe_based_heatmap_{paper_threshold}papers.csv", index_col=0)
for cat in category_names:
    if cat != "cancer":
        continue
    target_category = cat
    unique_probes = set(df[(df[target_category] != 0) & (df.drop(columns=[target_category]) == 0).all(axis=1)].index)
    g, l = get_gene_illumina(unique_probes)   
    cnum.append(len(set(corsiv_probe_df[corsiv_probe_df["probeId"].isin(unique_probes)]["corsiv_id"])))
    # print(f"{cat} ≥ {paper_threshold} papers: {len(g)} genes associated with {l} probes")
    cats.append(cat)
    thresh.append(paper_threshold)
    gnum.append(len(g))
    pnum.append(l)
    run_enrichr(g, output_fp=f"go_probe/go_results/manifest/{target_category}_all_{paper_threshold}papers_unique_0117.csv")
output_df = pd.DataFrame({"Categories": cats, "Paper cutoff": thresh, "Number of Genes": gnum, "Number of Unique Probes": pnum, "Number of CoRSIVs": cnum})

In [28]:
output_df

,Categories,Paper cutoff,Number of Genes,Number of Unique Probes,Number of CoRSIVs
0,cancer,1,8162,18132,152


In [ ]:
# print genes for enrichr-kg web
paper_threshold = 3
target_category = "neurological"

e = pd.read_csv("../humanData/EPIC_manifest_cleaned.csv")
hm = pd.read_csv("../humanData/HM450_manifest_cleaned.csv")
ref = pd.concat([e, hm])
ref.drop_duplicates(inplace=True)

df = pd.read_csv(f"heatmap/probe_based_heatmap_{paper_threshold}papers.csv", index_col=0)
unique_probes = set(df[(df[target_category] != 0) & (df.drop(columns=[target_category]) == 0).all(axis=1)].index)
g, l = get_gene_illumina(unique_probes)
for gg in g:
    print(gg)

In [ ]:
color_map = ["#ae3c60","#f3d83c", "#df473c","#f3c33c","#255e79","#267778"]
database_colors = dict(zip(target_libraries, color_map))
database_colors

In [ ]:
import matplotlib
paper_threshold = 3
for category in category_names:
    df = pd.read_csv(f"go_probe/go_results/{category}_all_{paper_threshold}papers_unique.csv")
    df = df[(df["Database"] == "Jensen_DISEASES") | (df["Database"] == "DisGeNET")]
    if df.empty:
        continue
    # Group by Database and get top 10 terms for each
    grouped = df.groupby('Database')
    top_10_per_db = [group.nsmallest(10, 'Adjusted p-value') for _, group in grouped]

    # Combine all top 10 lists and sort by adjusted p-value
    combined_top = pd.concat(top_10_per_db).sort_values('Adjusted p-value')

    # Calculate log of adjusted p-value
    log_p = -np.log10(combined_top['Adjusted p-value'])

    # Create a single figure
    fig, ax = plt.subplots(figsize=(8, len(combined_top) * 0.3))
    plt.rcParams['font.family'] = 'Helvetica'
    plt.rcParams['font.size'] = 10

    # Create horizontal bar plot with color coding
    bars = ax.barh(range(len(combined_top)), log_p, height=0.8, 
                color=[(*matplotlib.colors.to_rgb(database_colors[db]), 0.6) for db in combined_top['Database']])

    # Add labels to the bars
    for i, (_, row) in enumerate(combined_top.iterrows()):
        ax.text(0.05, i, f"{row['Name']}", ha='left', va='center', fontsize=10)

    # Customize the plot
    ax.set_title(f'Top 10 Terms from Each Database - {category.capitalize()}')
    ax.set_xlabel('-log10(Adjusted P-value)')
    ax.set_yticks(range(len(combined_top)))
    ax.set_yticklabels(combined_top['Database'])  # Set database names as y-axis labels
    ax.invert_yaxis()  # Invert y-axis to show highest significance at the top

    plt.tight_layout()
    plt.show()
    # plt.savefig(f"/Users/antata/Desktop/text-mining-figures/final/go/{category}_{paper_threshold}papers_all_databases.jpeg", format="jpeg")


In [ ]:

category = "neurological"
df = pd.read_csv(f"go_probe/go_results/{category}_all_3papers_unique.csv")


# Group by Database and sort within each group by adjusted p-value
grouped = df.groupby('Database')

fig, axs = plt.subplots(len(grouped), 1, figsize=(12, 6*len(grouped)), squeeze=False)
axs = axs.flatten()

for i, (database, group) in enumerate(grouped):
    # Sort by adjusted p-value and get top 10
    top_10 = group.sort_values('Adjusted p-value').head(10)
    
    # Calculate log of adjusted p-value
    log_p = -np.log10(top_10['Adjusted p-value'])
    
    # Create horizontal bar plot
    bars = axs[i].barh(top_10['Name'], log_p)
    
    # Add labels to the bars
    for bar in bars:
        width = bar.get_width()
        axs[i].text(width, bar.get_y() + bar.get_height()/2, 
                    f'{width:.2f}', ha='left', va='center')
    
    # Customize the plot
    axs[i].set_title(f'Top 10 Terms for {database}')
    axs[i].set_xlabel('-log10(Adjusted P-value)')
    axs[i].invert_yaxis()  # Invert y-axis to show highest significance at the top

plt.tight_layout()
plt.show()


In [ ]:
hm = pd.read_csv("../humanData/humanmethylation450_15017482_v1-2.csv", skiprows=7)
hm = hm[hm["IlmnID"].isin(hm450_probe_list)]
assert len(set(hm["IlmnID"])) == hm.shape[0]
hm = hm[["Name", 'UCSC_RefGene_Name', 'UCSC_RefGene_Accession',
       'UCSC_RefGene_Group', 'UCSC_CpG_Islands_Name',
       'Relation_to_UCSC_CpG_Island']]
hm2 = pd.read_csv("../humanData/database/HM450.hg38.txt", sep="\t", names=["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID"])
m = pd.merge(hm, hm2, left_on="Name", right_on="Probe_ID")
assert m.shape[0] == hm2.shape[0]
m = m[["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID", "UCSC_RefGene_Name", "UCSC_RefGene_Group"]]
m.to_csv("../humanData/HM450_manifest_cleaned.csv", index=0)

In [ ]:
e = pd.read_csv("../humanData/infinium-methylationepic-v-1-0-b5-manifest-file.csv", skiprows=7)
e = e[e["IlmnID"].isin(epic_probe_list)]
assert len(set(e["IlmnID"])) == e.shape[0]
e = e[["Name", 'UCSC_RefGene_Name', 'UCSC_RefGene_Group', 'CHR_hg38', 'Start_hg38', 'End_hg38']]
e2 = pd.read_csv("../humanData/database/EPIC.hg38.txt", sep="\t", names=["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID"])
m = pd.merge(e, e2, left_on="Name", right_on="Probe_ID")
assert m.shape[0] == e2.shape[0]
# m = m[["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID", "UCSC_RefGene_Name", "UCSC_RefGene_Group", 'CHR_hg38', 'Start_hg38', 'End_hg38']]
# m["Start_hg38"] = m["Start_hg38"].astype(int)
# m["End_hg38"] = m["End_hg38"].astype(int)
# m[m["Start_hg38"] != m["Probe_Start"]]
# m[m["End_hg38"] != m["Probe_End"]]
m = m[["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID", "UCSC_RefGene_Name", "UCSC_RefGene_Group"]]
m.to_csv("../humanData/EPIC_manifest_cleaned.csv", index=0)

In [ ]:
epic_hm450_probe = pd.read_csv("../control/humanData/EPIC_HM450.clean.bed", sep="\t", names=["chr", "start", "end", "probeId"])
epic_hm450_probe = epic_hm450_probe[epic_hm450_probe["probeId"].isin(corsiv_probe_list)]
df = epic_hm450_probe[epic_hm450_probe.duplicated(subset="probeId", keep=False)]
df.sort_values(by="probeId", inplace=True)
df

In [ ]:
# genic feature
e = pd.read_csv("../humanData/EPIC_manifest_cleaned.csv")
hm = pd.read_csv("../humanData/HM450_manifest_cleaned.csv")
ref = pd.concat([e, hm])
ref.drop_duplicates(inplace=True)
ref.loc[
    (ref["UCSC_RefGene_Name"].isna()) & (ref["UCSC_RefGene_Group"].isna()),
    "UCSC_RefGene_Group"
] = "Intergenic"
ref
m = pd.merge(ref, corsiv_probe_df[["chr", "start", "end", "probeId", "corsiv_id"]], left_on=["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID"], right_on=["chr", "start", "end", "probeId"])
m = m[["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID", "UCSC_RefGene_Name", "UCSC_RefGene_Group", "corsiv_id"]]
m.to_csv("../humanData/corsiv_manifest.csv", index=0)

In [3]:
# genic feature
e = pd.read_csv("../humanData/EPIC_manifest_cleaned.csv")
hm = pd.read_csv("../humanData/HM450_manifest_cleaned.csv")
ref = pd.concat([e, hm])
ref.drop_duplicates(inplace=True)
ref.loc[
    (ref["UCSC_RefGene_Name"].isna()) & (ref["UCSC_RefGene_Group"].isna()),
    "UCSC_RefGene_Group"
] = "Intergenic"
ref
m = pd.merge(ref, control_probe_df[["chr", "start", "end", "probeId", "id"]], left_on=["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID"], right_on=["chr", "start", "end", "probeId"])
m = m[["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID", "UCSC_RefGene_Name", "UCSC_RefGene_Group", "id"]]
m.to_csv("../humanData/control_manifest.csv", index=0)

In [ ]:
df = pd.read_csv("../humanData/corsiv_manifest.csv")
df["Unique Gene"] = df["UCSC_RefGene_Name"].apply(lambda x: ";".join(set(x.split(";"))) if isinstance(x, str) else "")
tmp = corsiv_probe_df[["chr", "corsiv_start", "corsiv_end", "corsiv_id"]]
tmp = tmp.drop_duplicates("corsiv_id")
df = pd.merge(df, tmp, on="corsiv_id")
df.rename(columns={"chr": "CoRSIV_Chr", "corsiv_start": "CoRSIV_Start", "corsiv_end": "CoRSIV_End", "corsiv_id": "CoRSIV_ID"}, inplace=True)
df = df[["Probe_Chr", "Probe_Start", "Probe_End", "Probe_ID", "Unique Gene", "CoRSIV_Chr", "CoRSIV_Start", "CoRSIV_End", "CoRSIV_ID"]]
# Create boolean columns for EPIC and HM450
e = pd.read_csv("../humanData/EPIC_manifest_cleaned.csv")
hm = pd.read_csv("../humanData/HM450_manifest_cleaned.csv")
e = e[e["Probe_ID"].isin(corsiv_probe_list)]
hm = hm[hm["Probe_ID"].isin(corsiv_probe_list)]
df['EPIC'] = df.apply(lambda row: tuple(row[['Probe_Chr', 'Probe_Start', 'Probe_End', 'Probe_ID']]) in set(map(tuple, e[['Probe_Chr', 'Probe_Start', 'Probe_End', 'Probe_ID']].values)), axis=1)
df['HM450'] = df.apply(lambda row: tuple(row[['Probe_Chr', 'Probe_Start', 'Probe_End', 'Probe_ID']]) in set(map(tuple, hm[['Probe_Chr', 'Probe_Start', 'Probe_End', 'Probe_ID']].values)), axis=1)
# sort by chr
df = df.sort_values(by=['Probe_Start', 'Probe_End'])
chr_order = {f'chr{i}': i for i in range(1, 23)}

# Sort the DataFrame using the custom chromosome order
df = df.sort_values(
    by=['Probe_Chr', 'Probe_Start'],
    key=lambda x: x.map(chr_order) if x.name == 'Probe_Chr' else x
)

df.to_csv("../humanData/corsiv_probe_manifest_annotated.csv", index=0)



In [ ]:
df1 = pd.read_csv("../humanData/corsiv_probe_manifest_annotated.csv")[["Probe_ID", "Unique Gene"]]
df2 = pd.read_excel("../gunasekara-2023.xlsx", sheet_name="S18")[["EPIC_probe_ID", "EPIC_gene"]]
df3 = pd.read_csv("/Users/antata/Downloads/EPIC.hg38.manifest.gencode.v36.tsv", header=0, sep="\t")
m = pd.merge(df1, df2, left_on="Probe_ID", right_on="EPIC_probe_ID")
df3 = df3[df3["probeID"].isin(m["EPIC_probe_ID"])][["probeID", "genesUniq"]]
m = pd.merge(m, df3, left_on="Probe_ID", right_on="probeID")
m.columns = ["Probe_ID", "manifest", "EPIC_probe_ID", "S18", "probeID", "gencodev22"]
m = m[["Probe_ID", "manifest", "S18", "gencodev22"]]
m
# m["S18"] = m["S18"].apply(lambda x: set(x.split(";")) if isinstance(x, str) else set([]))
# m["gencodev22"] = m["gencodev22"].apply(lambda x: set(x.split(";")) if isinstance(x, str) else set([]))
# m[m["S18"] != m["gencodev22"]]

In [ ]:
df = pd.read_csv("/Users/antata/Downloads/EPIC.hg38.manifest.gencode.v22.tsv", header=0, sep="\t")
df[df["probeID"].isin(m["EPIC_probe_ID"])][["probeID", "genesUniq"]]


In [ ]:
df = pd.read_csv("../humanData/corsiv_manifest.csv")

def parse_gene_group(row):
    if row["UCSC_RefGene_Group"] == "Intergenic":
        return pd.Series([set([("_", "Intergenic")]), set([("Intergenic")])])
    names = row["UCSC_RefGene_Name"].split(";")
    groups = row["UCSC_RefGene_Group"].split(";")
    gene_groups = set(zip(names, groups))
    group_set = set(groups)
    return pd.Series([gene_groups, group_set])

def cleanup(row):
    if len(row) > 1:
        row.discard("Intergenic")
    return row

df[["gene_groups", "groups"]] = df.apply(parse_gene_group, axis=1)
df = df[["corsiv_id", "Probe_ID", "gene_groups", "groups"]]
all_corsiv_df = df.groupby('corsiv_id')['groups'].apply(lambda x: set().union(*x)).reset_index()
all_corsiv_df["groups"] = all_corsiv_df["groups"].apply(cleanup)
illumina_covered_corsiv = all_corsiv_df.shape[0]
all_corsiv_tally = {}
for row in all_corsiv_df['groups']:
    for group in row:
        all_corsiv_tally[group] = all_corsiv_tally.get(group, 0) + 1
if "ExonBnd" in all_corsiv_tally:
    del all_corsiv_tally["ExonBnd"]

def tally_count(category, paper_threshold=3):
    c = read_in_probes(category)
    c = {k:v for k, v in c.items() if v >= paper_threshold}
    cat_df = df[df["Probe_ID"].isin(c)]
    cat_df = cat_df.groupby('corsiv_id')['groups'].apply(lambda x: set().union(*x)).reset_index()
    cat_df["groups"] = cat_df["groups"].apply(cleanup)
    target_corsiv_tally = {}
    for row in cat_df['groups']:
        for group in row:
            target_corsiv_tally[group] = target_corsiv_tally.get(group, 0) + 1
    if "ExonBnd" in target_corsiv_tally:
        del target_corsiv_tally["ExonBnd"]
    return target_corsiv_tally, cat_df.shape[0]

def plot_gene_hist(cat, all, specific, count, output_path=None):
    keys = sorted(all.keys())
    s = [specific.get(key,0) for key in keys]
    a = [all[key] for key in keys]
    s_total = sum(s)
    all_total = sum(a)
    s_pct = [value / s_total * 100 for value in s]
    a_pct = [value / all_total * 100 for value in a]

    # Plotting
    bar_width = 0.35
    index = np.arange(len(keys))

    plt.figure(figsize=(12, 8))

    # Bar chart for CORSIV tally
    plt.bar(index, a_pct, bar_width, color='#55a868', label=f'All CoRSIVs ({illumina_covered_corsiv})')
    plt.bar(index+bar_width, s_pct, bar_width, color='#4c72b0', label=f'{cat.capitalize()} CoRSIVs ({count})')

    # Add labels and title
    plt.xlabel('Gene Region')
    plt.ylabel('Percentage (%)')
    plt.title(f'All CoRSIVs vs CoRSIVs Showed up in {cat.capitalize()} Papers')

    # Add xticks and legend
    plt.xticks(index + bar_width / 2, keys)
    plt.legend()
    if output_path == None:
        plt.show()
    else:
        plt.savefig(output_path, format="jpeg")
    plt.close()
    return

    

In [ ]:
for name in category_names:
    d, count = tally_count(name)
    output_path = f"/Users/antata/Desktop/text-mining-figures/final/gene_hist/{name}_histograms.jpeg"
    plot_gene_hist(name, all_corsiv_tally, d, count, None)
    
    

In [ ]:
df

In [ ]:
corsiv_probe_df[["probeId", "corsiv_id"]]